In [6]:
%env PIRATE_WEATHER_API_KEY=
%env OPENAI_API_KEY=

env: PIRATE_WEATHER_API_KEY=
env: OPENAI_API_KEY=


In [12]:
%pip install function_calling_tools

  Attempting uninstall: function-calling-tools
    Found existing installation: function-calling-tools 0.1.0
    Uninstalling function-calling-tools-0.1.0:
      Successfully uninstalled function-calling-tools-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import openai
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from function_calling_tools.pirateweather import WeatherFetcher

tools = [
  WeatherFetcher(),
]

memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

agent_kwargs = {
  "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}

prompt = """
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
"""

memory.save_context({"input": prompt}, {"output": "I got it!"})

openai.api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)

In [5]:
agent.run(input="What is the weather like in Tokyo today?")



> Entering new AgentExecutor chain...

Invoking: `fetch_weather_info` with `{'latitude': 35.6895, 'longitude': 139.6917}`


{'latitude': 35.6895, 'longitude': 139.6917, 'timezone': 'Etc/GMT-9', 'offset': 9.0, 'elevation': 41, 'currently': {'time': 1694402520, 'summary': 'Cloudy', 'icon': 'cloudy', 'nearestStormDistance': 0, 'nearestStormBearing': 0, 'precipIntensity': 0.0362, 'precipProbability': 0.03, 'precipIntensityError': 0.05, 'precipType': 'rain', 'temperature': 30.07, 'apparentTemperature': 38.4, 'dewPoint': 20.89, 'humidity': 0.59, 'pressure': 1009.5, 'windSpeed': 4.07, 'windGust': 4.88, 'windBearing': 177, 'cloudCover': 1.0, 'uvIndex': 4.97, 'visibility': 16.09, 'ozone': 280.95}, 'daily': {'summary': 'Clear', 'icon': 'clear-day', 'data': [{'time': 1694358000, 'icon': 'cloudy', 'summary': 'Cloudy', 'sunriseTime': 1694377229, 'sunsetTime': 1694422510, 'moonPhase': 0.9, 'precipIntensity': 0.046, 'precipIntensityMax': 0.0913, 'precipIntensityMaxTime': 1694412000, 'precipProbabil

'The weather in Tokyo today is cloudy. The temperature is 30.07°C with a humidity of 59%. The wind speed is 4.07 km/h coming from the south. The visibility is 16.09 km.'